<a href="https://colab.research.google.com/github/jinzzasol/Mathsage/blob/main/Mathsage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [2]:
# import os
# os.chdir('drive/MyDrive/CS5624_NLP/MathSage')

In [3]:
# !pip install -q transformers evaluate

In [4]:
# # import json
# import pandas as pd
# import glob

In [5]:
# f = open('MathQA_DB/train.json')
# data = json.load(f)
# data[0]

In [6]:
import pandas as pd
import glob
import os
from datasets import Dataset, load_dataset

# Load Data
Create `pandas dataframe` from `JSON` files


In [7]:
def csv_to_dataframe():
  # Load all the csv files into pandas dataframes
  all_files = glob.glob('Connor/dataset/*.csv')
  data_dict = {}
  for filename in all_files:
      df = pd.read_csv(filename)
      key = os.path.basename(filename).split(".")[0]
      data_dict[key] = df

  return data_dict

In [8]:
mathqa_db = csv_to_dataframe()

In [9]:
mathqa_db.keys()

dict_keys(['test', 'train', 'validation'])

In [10]:
train_df = mathqa_db['train']
train_df.head()

,problem,category,solution,formula,formula_no_const,annotated_formula,incremental,nums,ops,incremental_no_const,nums_no_const,ops_no_const,tree
0,the banker ' s gain of a certain sum due 3 yea...,gain,400.0,((const_100 * ((36.0 * const_100) / (3.0 * 10....,((100 * ((36.0 * 100) / (3.0 * 10.0))) / (3.0 ...,"divide(multiply(const_100, divide(multiply(36,...","['36*const_100', '3*10', None, None, None, Non...","{'10', '36', 'const_100', '3'}","{'*', '/'}","['36*100', '3*10', None, None, None, None] ; [...","{'10', '36', '3', '100'}","{'*', '/'}","{'name': '/', 'children': [{'name': '*', 'chil..."
1,sophia finished 2 / 3 of a book . she calculat...,general,270.0,(90.0 / (const_1 - (2.0 / 3.0))),(90.0 / (1 - (2.0 / 3.0))),"divide(90, subtract(const_1, divide(2, 3)))","['2/3', None, None, None, None, None] ; ['cons...","{'2', '90', '3', 'const_1'}","{'/', '-'}","['2/3', None, None, None, None, None] ; ['1-va...","{'3', '2', '90', '1'}","{'/', '-'}","{'name': '/', 'children': [{'name': '90'}, {'n..."
2,120 is what percent of 50 ?,gain,240.0,((120.0 / 50.0) * const_100),((120.0 / 50.0) * 100),"multiply(divide(120, 50), const_100)","['120/50', None, None, None, None, None] ; ['v...","{'const_100', '50', '120'}","{'*', '/'}","['120/50', None, None, None, None, None] ; ['v...","{'100', '50', '120'}","{'*', '/'}","{'name': '*', 'children': [{'name': '/', 'chil..."
3,there are 10 girls and 20 boys in a classroom ...,other,0.5,(10.0 / 20.0),(10.0 / 20.0),"divide(10, 20)","['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"{'name': '/', 'children': [{'name': '10'}, {'n..."
4,an empty fuel tank with a capacity of 218 gall...,gain,122.0,(((218.0 * (16.0 / const_100)) - 30.0) / ((16....,(((218.0 * (16.0 / 100)) - 30.0) / ((16.0 / 10...,"divide(subtract(multiply(218, divide(16, const...","['16/const_100', None, None, None, None, None]...","{'218', '12', 'const_100', '16', '30'}","{'*', '/', '-'}","['16/100', None, None, None, None, None] ; ['2...","{'218', '12', '100', '16', '30'}","{'*', '/', '-'}","{'name': '/', 'children': [{'name': '-', 'chil..."


In [11]:
train_df[train_df.duplicated()]

,problem,category,solution,formula,formula_no_const,annotated_formula,incremental,nums,ops,incremental_no_const,nums_no_const,ops_no_const,tree


In [12]:
# test_df = mathqa_db['test']
# test_df.head()

In [13]:
# test_df[test_df.duplicated()]

Model adn tokenizer

In [14]:
# Pytorch Version is available via Hugging Face Transformers
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

TEST_MODEL = r'tbs17/MathBERT-custom'

# Download the MathBERT -basevocab
# tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT")
# model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT")

# Download the MathBERT -mathvocab
tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT-custom")
model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT-custom")

Some weights of the model checkpoint at tbs17/MathBERT-custom were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Batch Generator like

https://github.com/NeuraSearch/NeurIPS-2022-Submission-3358/blob/main/models/batch_generator.py

In [15]:
train = train_df.copy()

In [16]:
def tokenization(items):
    tokenized = tokenizer(items['text'], max_length = 256, padding = 'max_length', truncation=True)
    return tokenized

def tokenization_labels(items):
    tokenized = tokenizer(items['text'], max_length = 256, padding = 'max_length', truncation=True)
    tokenized['label'] = tokenizer(items['label'], max_length = 256, padding = 'max_length', truncation=True)['input_ids']
    return tokenized

In [17]:
train['question_passage_tokens'] = train['problem'].apply(lambda x: x.split(' '))
train.head()

,problem,category,solution,formula,formula_no_const,annotated_formula,incremental,nums,ops,incremental_no_const,nums_no_const,ops_no_const,tree,question_passage_tokens
0,the banker ' s gain of a certain sum due 3 yea...,gain,400.0,((const_100 * ((36.0 * const_100) / (3.0 * 10....,((100 * ((36.0 * 100) / (3.0 * 10.0))) / (3.0 ...,"divide(multiply(const_100, divide(multiply(36,...","['36*const_100', '3*10', None, None, None, Non...","{'10', '36', 'const_100', '3'}","{'*', '/'}","['36*100', '3*10', None, None, None, None] ; [...","{'10', '36', '3', '100'}","{'*', '/'}","{'name': '/', 'children': [{'name': '*', 'chil...","[the, banker, ', s, gain, of, a, certain, sum,..."
1,sophia finished 2 / 3 of a book . she calculat...,general,270.0,(90.0 / (const_1 - (2.0 / 3.0))),(90.0 / (1 - (2.0 / 3.0))),"divide(90, subtract(const_1, divide(2, 3)))","['2/3', None, None, None, None, None] ; ['cons...","{'2', '90', '3', 'const_1'}","{'/', '-'}","['2/3', None, None, None, None, None] ; ['1-va...","{'3', '2', '90', '1'}","{'/', '-'}","{'name': '/', 'children': [{'name': '90'}, {'n...","[sophia, finished, 2, /, 3, of, a, book, ., sh..."
2,120 is what percent of 50 ?,gain,240.0,((120.0 / 50.0) * const_100),((120.0 / 50.0) * 100),"multiply(divide(120, 50), const_100)","['120/50', None, None, None, None, None] ; ['v...","{'const_100', '50', '120'}","{'*', '/'}","['120/50', None, None, None, None, None] ; ['v...","{'100', '50', '120'}","{'*', '/'}","{'name': '*', 'children': [{'name': '/', 'chil...","[120, is, what, percent, of, 50, ?]"
3,there are 10 girls and 20 boys in a classroom ...,other,0.5,(10.0 / 20.0),(10.0 / 20.0),"divide(10, 20)","['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"{'name': '/', 'children': [{'name': '10'}, {'n...","[there, are, 10, girls, and, 20, boys, in, a, ..."
4,an empty fuel tank with a capacity of 218 gall...,gain,122.0,(((218.0 * (16.0 / const_100)) - 30.0) / ((16....,(((218.0 * (16.0 / 100)) - 30.0) / ((16.0 / 10...,"divide(subtract(multiply(218, divide(16, const...","['16/const_100', None, None, None, None, None]...","{'218', '12', 'const_100', '16', '30'}","{'*', '/', '-'}","['16/100', None, None, None, None, None] ; ['2...","{'218', '12', '100', '16', '30'}","{'*', '/', '-'}","{'name': '/', 'children': [{'name': '-', 'chil...","[an, empty, fuel, tank, with, a, capacity, of,..."


In [18]:
train['question_passage_tokens_ids'] = train['question_passage_tokens'].apply(lambda x: [tokenizer.cls_token] + x + [tokenizer.sep_token])
train.head()

,problem,category,solution,formula,formula_no_const,annotated_formula,incremental,nums,ops,incremental_no_const,nums_no_const,ops_no_const,tree,question_passage_tokens,question_passage_tokens_ids
0,the banker ' s gain of a certain sum due 3 yea...,gain,400.0,((const_100 * ((36.0 * const_100) / (3.0 * 10....,((100 * ((36.0 * 100) / (3.0 * 10.0))) / (3.0 ...,"divide(multiply(const_100, divide(multiply(36,...","['36*const_100', '3*10', None, None, None, Non...","{'10', '36', 'const_100', '3'}","{'*', '/'}","['36*100', '3*10', None, None, None, None] ; [...","{'10', '36', '3', '100'}","{'*', '/'}","{'name': '/', 'children': [{'name': '*', 'chil...","[the, banker, ', s, gain, of, a, certain, sum,...","[[CLS], the, banker, ', s, gain, of, a, certai..."
1,sophia finished 2 / 3 of a book . she calculat...,general,270.0,(90.0 / (const_1 - (2.0 / 3.0))),(90.0 / (1 - (2.0 / 3.0))),"divide(90, subtract(const_1, divide(2, 3)))","['2/3', None, None, None, None, None] ; ['cons...","{'2', '90', '3', 'const_1'}","{'/', '-'}","['2/3', None, None, None, None, None] ; ['1-va...","{'3', '2', '90', '1'}","{'/', '-'}","{'name': '/', 'children': [{'name': '90'}, {'n...","[sophia, finished, 2, /, 3, of, a, book, ., sh...","[[CLS], sophia, finished, 2, /, 3, of, a, book..."
2,120 is what percent of 50 ?,gain,240.0,((120.0 / 50.0) * const_100),((120.0 / 50.0) * 100),"multiply(divide(120, 50), const_100)","['120/50', None, None, None, None, None] ; ['v...","{'const_100', '50', '120'}","{'*', '/'}","['120/50', None, None, None, None, None] ; ['v...","{'100', '50', '120'}","{'*', '/'}","{'name': '*', 'children': [{'name': '/', 'chil...","[120, is, what, percent, of, 50, ?]","[[CLS], 120, is, what, percent, of, 50, ?, [SEP]]"
3,there are 10 girls and 20 boys in a classroom ...,other,0.5,(10.0 / 20.0),(10.0 / 20.0),"divide(10, 20)","['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"{'name': '/', 'children': [{'name': '10'}, {'n...","[there, are, 10, girls, and, 20, boys, in, a, ...","[[CLS], there, are, 10, girls, and, 20, boys, ..."
4,an empty fuel tank with a capacity of 218 gall...,gain,122.0,(((218.0 * (16.0 / const_100)) - 30.0) / ((16....,(((218.0 * (16.0 / 100)) - 30.0) / ((16.0 / 10...,"divide(subtract(multiply(218, divide(16, const...","['16/const_100', None, None, None, None, None]...","{'218', '12', 'const_100', '16', '30'}","{'*', '/', '-'}","['16/100', None, None, None, None, None] ; ['2...","{'218', '12', '100', '16', '30'}","{'*', '/', '-'}","{'name': '/', 'children': [{'name': '-', 'chil...","[an, empty, fuel, tank, with, a, capacity, of,...","[[CLS], an, empty, fuel, tank, with, a, capaci..."


In [19]:
import re

def extract_numbers(problem):
    return re.findall(r'\d+', problem)

arguments = train['problem'].apply(extract_numbers)
arguments
train['arguments'] = arguments

In [20]:
question_passage_ids = train['question_passage_tokens_ids'].apply(tokenizer.convert_tokens_to_ids)
question_index = train['question_passage_tokens_ids'].apply(lambda x: x.index(tokenizer.sep_token) if tokenizer.sep_token in x else -1)

train['question_passage_ids'] = question_passage_ids
train['question_index'] = question_index

In [21]:
import re

def get_number_indices(tokens):
    number_indices = []
    for i, token in enumerate(tokens):
        if re.match(r'^\d+(\.\d+)?$', token):
            number_indices.append(i)
    return number_indices

argument_indices_offset = train['question_passage_tokens_ids'].apply(get_number_indices)
argument_indices_offset

0                      [11, 15, 22]
1                        [3, 5, 15]
2                            [1, 6]
3                            [3, 6]
4                   [9, 28, 37, 49]
                    ...            
18316                   [3, 12, 25]
18317    [2, 5, 15, 25, 30, 39, 48]
18318               [2, 21, 24, 41]
18319                    [3, 6, 20]
18320                    [4, 7, 10]
Name: question_passage_tokens_ids, Length: 18321, dtype: object

In [22]:
train['argument_indices_offset'] = argument_indices_offset

In [23]:
train['number_in_qp_mask'] = train['argument_indices_offset'].apply(lambda x: [0 if i not in x else 1 for i in range(len(question_passage_ids))])
number_in_qp_mask = train['number_in_qp_mask']

In [24]:
train['number_mask'] = train['question_passage_tokens'].apply(lambda x: [1 if i in get_number_indices(x) else 0 for i in range(len(x))])
train['number_mask']

0        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
1        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
2                                    [1, 0, 0, 0, 0, 1, 0]
3        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
                               ...                        
18316    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
18317    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
18318    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
18319    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
18320                       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1]
Name: number_mask, Length: 18321, dtype: object

In [25]:
import re
from configurations import OPERATION_LIST_MATHQA, OPERATION_LIST_MATHQA_2, CONST_LIST_MATHQA

operation_list = train['annotated_formula'].apply(lambda x: [op for op in OPERATION_LIST_MATHQA if op in x])
print(operation_list)
train['golden_op'] = operation_list

0                  [multiply, divide]
1                  [subtract, divide]
2                  [multiply, divide]
3                            [divide]
4        [subtract, multiply, divide]
                     ...             
18316    [subtract, multiply, divide]
18317              [multiply, divide]
18318              [subtract, divide]
18319       [add, subtract, multiply]
18320           [add, divide, negate]
Name: annotated_formula, Length: 18321, dtype: object


In [26]:
op_list = OPERATION_LIST_MATHQA

golden_op_ids = train['golden_op'].apply(lambda ops: [op_list.index(op) for op in ops])
print(golden_op_ids)
train['golden_op_ids'] = golden_op_ids

0            [2, 3]
1            [1, 3]
2            [2, 3]
3               [3]
4         [1, 2, 3]
            ...    
18316     [1, 2, 3]
18317        [2, 3]
18318        [1, 3]
18319     [0, 1, 2]
18320    [0, 3, 62]
Name: golden_op, Length: 18321, dtype: object


In [27]:
train.head()

,problem,category,solution,formula,formula_no_const,annotated_formula,incremental,nums,ops,incremental_no_const,...,question_passage_tokens,question_passage_tokens_ids,arguments,question_passage_ids,question_index,argument_indices_offset,number_in_qp_mask,number_mask,golden_op,golden_op_ids
0,the banker ' s gain of a certain sum due 3 yea...,gain,400.0,((const_100 * ((36.0 * const_100) / (3.0 * 10....,((100 * ((36.0 * 100) / (3.0 * 10.0))) / (3.0 ...,"divide(multiply(const_100, divide(multiply(36,...","['36*const_100', '3*10', None, None, None, Non...","{'10', '36', 'const_100', '3'}","{'*', '/'}","['36*100', '3*10', None, None, None, None] ; [...",...,"[the, banker, ', s, gain, of, a, certain, sum,...","[[CLS], the, banker, ', s, gain, of, a, certai...","[3, 10, 36]","[101, 1996, 13448, 1005, 1055, 5114, 1997, 103...",30,"[11, 15, 22]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[multiply, divide]","[2, 3]"
1,sophia finished 2 / 3 of a book . she calculat...,general,270.0,(90.0 / (const_1 - (2.0 / 3.0))),(90.0 / (1 - (2.0 / 3.0))),"divide(90, subtract(const_1, divide(2, 3)))","['2/3', None, None, None, None, None] ; ['cons...","{'2', '90', '3', 'const_1'}","{'/', '-'}","['2/3', None, None, None, None, None] ; ['1-va...",...,"[sophia, finished, 2, /, 3, of, a, book, ., sh...","[[CLS], sophia, finished, 2, /, 3, of, a, book...","[2, 3, 90]","[101, 9665, 2736, 1016, 1013, 1017, 1997, 1037...",31,"[3, 5, 15]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[subtract, divide]","[1, 3]"
2,120 is what percent of 50 ?,gain,240.0,((120.0 / 50.0) * const_100),((120.0 / 50.0) * 100),"multiply(divide(120, 50), const_100)","['120/50', None, None, None, None, None] ; ['v...","{'const_100', '50', '120'}","{'*', '/'}","['120/50', None, None, None, None, None] ; ['v...",...,"[120, is, what, percent, of, 50, ?]","[[CLS], 120, is, what, percent, of, 50, ?, [SEP]]","[120, 50]","[101, 6036, 2003, 2054, 3867, 1997, 2753, 1029...",8,"[1, 6]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0]","[multiply, divide]","[2, 3]"
3,there are 10 girls and 20 boys in a classroom ...,other,0.5,(10.0 / 20.0),(10.0 / 20.0),"divide(10, 20)","['10/20', None, None, None, None, None]","{'10', '20'}",{'/'},"['10/20', None, None, None, None, None]",...,"[there, are, 10, girls, and, 20, boys, in, a, ...","[[CLS], there, are, 10, girls, and, 20, boys, ...","[10, 20]","[101, 2045, 2024, 2184, 3057, 1998, 2322, 3337...",21,"[3, 6]","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[divide],[3]
4,an empty fuel tank with a capacity of 218 gall...,gain,122.0,(((218.0 * (16.0 / const_100)) - 30.0) / ((16....,(((218.0 * (16.0 / 100)) - 30.0) / ((16.0 / 10...,"divide(subtract(multiply(218, divide(16, const...","['16/const_100', None, None, None, None, None]...","{'218', '12', 'const_100', '16', '30'}","{'*', '/', '-'}","['16/100', None, None, None, None, None] ; ['2...",...,"[an, empty, fuel, tank, with, a, capacity, of,...","[[CLS], an, empty, fuel, tank, with, a, capaci...","[218, 12, 16, 30]","[101, 2019, 4064, 4762, 4951, 2007, 1037, 3977...",63,"[9, 28, 37, 49]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[subtract, multiply, divide]","[1, 2, 3]"


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18321 entries, 0 to 18320
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   problem                      18321 non-null  object 
 1   category                     18321 non-null  object 
 2   solution                     18321 non-null  float64
 3   formula                      18321 non-null  object 
 4   formula_no_const             18321 non-null  object 
 5   annotated_formula            18321 non-null  object 
 6   incremental                  18275 non-null  object 
 7   nums                         18321 non-null  object 
 8   ops                          18321 non-null  object 
 9   incremental_no_const         18275 non-null  object 
 10  nums_no_const                18321 non-null  object 
 11  ops_no_const                 18321 non-null  object 
 12  tree                         18321 non-null  object 
 13  question_passage

In [29]:
# train = train.iloc[:, 13:] # drop columns from original train
# train.info()

In [30]:
max_op_len = 0
gold_op_len = len(golden_op_ids)
if gold_op_len > max_op_len:
    max_op_len = gold_op_len
max_op_len

18321

In [31]:
const_list = CONST_LIST_MATHQA

def convert_argu_to_ids(golden_argument, 
                        arguments, 
                        argument_indices_offset,
                        id=None):
    golden_argument_ids = []
    for sub_argu in golden_argument:
        sub_argu_ids = []
        for argu in sub_argu:
            if argu in const_list:
                sub_argu_ids.append(const_list.index(argu))
            else:
                idx_for_indices = int(argu.replace("n", ""))
                assert idx_for_indices < len(arguments)
                argu_ids = argument_indices_offset[idx_for_indices] + len(const_list)
                sub_argu_ids.append(argu_ids)
        golden_argument_ids.append(sub_argu_ids)
    

In [32]:
# golden_argument_ids = convert_argu_to_ids(train['golden_argument'], # there is no golden argument
#                                           train['arguments'],
#                                           train['argument_indices_offset'],
#                                           train['id'],
#                                           )

In [33]:
units_token = [
                tokenizer.tokenize("<pad>"),
                tokenizer.tokenize("ousand"),
                tokenizer.tokenize("million"),
                tokenizer.tokenize("billion"),
                tokenizer.tokenize("percent"),
                tokenizer.tokenize("year")
            ]
units_idx = [tokenizer.convert_tokens_to_ids(token)[0] for token in units_token]

In [34]:
datas = []
datas.append((question_passage_ids, 
                question_index, 
                None, # number_unit_graph_mask, 
                None, # number_number_graph_mask, 
                None, # number_indices_offset,
                number_in_qp_mask, 
                golden_op_ids, 
                None, # golden_argument_ids,
                units_idx, 
                None, # golden_sequence_program_ids,
                train))

In [35]:
datas

[(0        [101, 1996, 13448, 1005, 1055, 5114, 1997, 103...
  1        [101, 9665, 2736, 1016, 1013, 1017, 1997, 1037...
  2        [101, 6036, 2003, 2054, 3867, 1997, 2753, 1029...
  3        [101, 2045, 2024, 2184, 3057, 1998, 2322, 3337...
  4        [101, 2019, 4064, 4762, 4951, 2007, 1037, 3977...
                                 ...                        
  18316    [101, 1037, 3345, 8148, 1049, 2146, 5235, 1037...
  18317    [101, 2065, 1023, 5209, 16678, 2484, 12046, 11...
  18318    [101, 2065, 2385, 3867, 1997, 1996, 2493, 2012...
  18319    [101, 2779, 1997, 2184, 3503, 2003, 3590, 1010...
  18320    [101, 9611, 2917, 3160, 1016, 1060, 1009, 1015...
  Name: question_passage_tokens_ids, Length: 18321, dtype: object,
  0        30
  1        31
  2         8
  3        21
  4        63
           ..
  18316    34
  18317    54
  18318    57
  18319    23
  18320    11
  Name: question_passage_tokens_ids, Length: 18321, dtype: int64,
  None,
  None,
  None,
  0        [0, 0, 

In [38]:
import functools
from torch.utils.data import DataLoader

def data_loader(use_cuda):
    collate_fn = functools.partial(
        create_batch_data,
        pad_ids=tokenizer.pad_token_id,
        eof_ids=op_list.index("EOF"),
        argu_pad_ids=const_list.index("none"),
        max_input_len=max_position_embeddings,
        max_op_len=max_op_len,
        max_op_argu_len=max_op_argu_len,
        max_prog_len=max_prog_len,
        offset=offset,
        use_cuda=use_cuda,
    )

    loader = DataLoader(
        datas,
        batch_size=8,
        shuffle=True,
        collate_fn=collate_fn,
    )

    return loader

In [ ]:
# args
PLM=roberta-large
# PLM=t5-small
N_LAYERS=4
DROPOUT_P=0.1
MAX_OP_LEN=7
MAX_ARGU_LEN=2
MERGE_OP=0
N_HEAD=8
IS_PROGRAM_AS_SEQUENCE=0

MAX_EPOCH=50
T_BSZ=10
E_BSZ=10
GRADIENT_ACCUMULATION_STEPS=1
LR=1e-5
WD=1e-5
FINE_TUNE=1
SCHEDULED_SAMPLING=0
SAMPLING_K=300

LOG_PER_UPDATES=10
SAVE_EVERY_STEPS=1000

In [ ]:
max_op_len = MAX_OP_LEN
gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS
max_epoch = MAX_EPOCH

num_train_steps = int(len(train_data_loader) / gradient_accumulation_steps) * max_epoch

In [39]:
bsz = len(question_passage_ids_set) # batch size?

NameError: name 'question_passage_ids_set' is not defined

In [42]:
for batch in data_loader(use_cuda=False):
    pass

NameError: name 'max_position_embeddings' is not defined

In [13]:
# Split the *train* dataset into train, validation, test
from sklearn.model_selection import train_test_split

train_df = train_df.rename(columns={'problem':'text', 'formula':'label'})

# 80% - train, 10% - validation, 10% - test
# First, create 20% of the test dataset, then split it into validation and test dataset
train, test = train_test_split(train_df, test_size=0.2, shuffle=False)
val, test = train_test_split(test, test_size=0.5, shuffle=True)

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

train.shape, val.shape, test.shape,

((14656, 13), (1832, 13), (1833, 13))

# Load MathBert pretrained model

Use MathBert from Jia Tracy Shen, Michiharu Yamashita, Ethan Prihar, Neil Heffernan, Xintao Wu, Ben Graff, Dongwon Lee (2023) "MathBERT: A Pre-trained Language Model for General NLP Tasks in Mathematics Education". . doi:[10.48550/arXiv.2106.07340](https://doi.org/10.48550/arXiv.2106.07340)

In [14]:
# Create new index
train_idx = [i for i in range(len(train.index))]
test_idx = [i for i in range(len(test.index))]
val_idx = [i for i in range(len(val.index))]

# Convert to numpy
x_train = train['text'].values[train_idx]
x_test = test['text'].values[test_idx]
x_val = val['text'].values[val_idx]

y_train = train['label'].values[train_idx]
y_test = test['label'].values[test_idx]
y_val = val['label'].values[val_idx]


In [15]:
train = Dataset.from_pandas(train, preserve_index=False)
val = Dataset.from_pandas(val, preserve_index=False)
test = Dataset.from_pandas(test, preserve_index=False)

In [16]:
# Pytorch Version is available via Hugging Face Transformers
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

TEST_MODEL = r'tbs17/MathBERT-custom'

# Download the MathBERT -basevocab
# tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT")
# model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT")

# Download the MathBERT -mathvocab
tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT-custom")
model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT-custom")

Some weights of the model checkpoint at tbs17/MathBERT-custom were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
def tokenization(items):
    tokenized = tokenizer(items['text'], max_length = 256, padding = 'max_length', truncation=True)
    return tokenized

def tokenization_labels(items):
    tokenized = tokenizer(items['text'], max_length = 256, padding = 'max_length', truncation=True)
    tokenized['label'] = tokenizer(items['label'], max_length = 256, padding = 'max_length', truncation=True)['input_ids']
    return tokenized

In [18]:
train = train.map(tokenization, batched=True)

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

In [19]:
val = val.map(tokenization, batched=True)

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

In [20]:
test = test.map(tokenization_labels, batched=True)

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

In [4]:
from transformers import pipeline, TrainingArguments, Trainer, AutoModelForQuestionAnswering
from trl import SFTTrainer
import evaluate
import numpy as np

In [22]:
metric = evaluate.load('f1')

def compute_metrics(pred):
    output, labels = pred
    return metric.compute(predictions=np.argmax(output, axis=-1), references=labels)

args = TrainingArguments(
    output_dir='/Code/data/MathQA_DB/temp',
    evaluation_strategy="epoch", # Reports the metric after each epoch
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
)

trainer = SFTTrainer(
    model = TEST_MODEL,
    args = args,
    train_dataset = train,
    eval_dataset = val,
    tokenizer=tokenizer,
    max_seq_length=256,
    dataset_text_field='text',
)

d:\Anaconda3\envs\pytorch\lib\site-packages\trl\trainer\sft_trainer.py:125: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

In [23]:
import torch
torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
trainer.train()

  0%|          | 0/5496 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.8319, 'learning_rate': 4.5451237263464344e-05, 'epoch': 0.27}
{'loss': 0.0425, 'learning_rate': 4.090247452692867e-05, 'epoch': 0.55}
{'loss': 0.0193, 'learning_rate': 3.6353711790393013e-05, 'epoch': 0.82}


  0%|          | 0/229 [00:00<?, ?it/s]

{'eval_loss': 0.006190407555550337, 'eval_runtime': 12.514, 'eval_samples_per_second': 146.396, 'eval_steps_per_second': 18.299, 'epoch': 1.0}
{'loss': 0.0107, 'learning_rate': 3.1804949053857355e-05, 'epoch': 1.09}
{'loss': 0.0049, 'learning_rate': 2.725618631732169e-05, 'epoch': 1.36}
{'loss': 0.0038, 'learning_rate': 2.2707423580786028e-05, 'epoch': 1.64}
{'loss': 0.0035, 'learning_rate': 1.8158660844250363e-05, 'epoch': 1.91}


  0%|          | 0/229 [00:00<?, ?it/s]

{'eval_loss': 0.0029049436561763287, 'eval_runtime': 13.8125, 'eval_samples_per_second': 132.634, 'eval_steps_per_second': 16.579, 'epoch': 2.0}
{'loss': 0.0022, 'learning_rate': 1.3609898107714703e-05, 'epoch': 2.18}
{'loss': 0.0014, 'learning_rate': 9.06113537117904e-06, 'epoch': 2.46}
{'loss': 0.0008, 'learning_rate': 4.512372634643377e-06, 'epoch': 2.73}


  0%|          | 0/229 [00:00<?, ?it/s]

{'eval_loss': 0.002342228777706623, 'eval_runtime': 12.5223, 'eval_samples_per_second': 146.299, 'eval_steps_per_second': 18.287, 'epoch': 3.0}
{'train_runtime': 1071.9564, 'train_samples_per_second': 41.017, 'train_steps_per_second': 5.127, 'train_loss': 0.0838296020964738, 'epoch': 3.0}


TrainOutput(global_step=5496, training_loss=0.0838296020964738, metrics={'train_runtime': 1071.9564, 'train_samples_per_second': 41.017, 'train_steps_per_second': 5.127, 'train_loss': 0.0838296020964738, 'epoch': 3.0})

In [25]:
trainer.save_model('/Code/data/MathBert_test')

In [3]:
from transformers import pipeline
import re

# Load the fine-tuned model
model = pipeline('text2text-generation', model='/Code/data/MathBert_test', tokenizer='tbs17/MathBERT-custom')

# Provide a math problem to the model
problem = "What is the sum of 2 and 3?"
result = model(problem, max_length=32, do_sample=False)

# Extract the formula from the generated text
formula = result[0]['generated_text'].strip()

# Check if the formula is valid
if re.match(r'^\d+(\s*[+\-*/]\s*\d+)*$', formula):
    print(f'The model converted the problem "{problem}" to the formula "{formula}"')
else:
    print(f'The model failed to convert the problem "{problem}" to a valid formula')


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
The model 'BertLMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


The model failed to convert the problem "What is the sum of 2 and 3?" to a valid formula


## Test fine-tuned model

THe kernel kept crashing; Use Mathsage_test instead.

In [26]:
from transformers import pipeline

# Load the fine-tuned model
model = pipeline('text2text-generation', model='/Code/data/MathBert_test', tokenizer='tbs17/MathBERT-custom')

# Provide a math problem to the model
problem = "What is the sum of 2 and 3?"
result = model(problem, max_length=32, do_sample=False)

# Extract the answer from the generated text
answer = result
print(answer)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


: 